In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet, Lasso, Ridge


In [3]:
def canal_adaptive_elastic_net_online(X, y, beta_init, lambda1, lambda2, gamma, zeta, eta, iterations):
    n, d = X.shape
    beta = beta_init.copy()

    # Initializing dynamic ε and δ
    epsilon = zeta * np.mean(np.abs(y))
    delta = gamma * np.mean(np.abs(y))

    for t in range(iterations):
        for i in range(n):
            xi, yi = X[i], y[i]
            y_pred = xi @ beta
            error = yi - y_pred

           # Determine the gradient based on the error range
            if epsilon <= abs(error) < epsilon + delta:
                # Compute gradient according to specified complex condition
                grad = (-xi * np.sign(error) + 2 * lambda2 * beta +
                        lambda1 * ((-gamma) * np.abs(beta)**(-gamma - 1)) * np.sign(beta) * np.abs(beta) + np.abs(beta)**(-gamma) * np.sign(beta))
            else:
                # Compute gradient for the regular case
                grad = (2 * lambda2 * beta +
                        lambda1 * ((-gamma) * np.abs(beta)**(-gamma - 1)) * np.sign(beta) * np.abs(beta) + np.abs(beta)**(-gamma) * np.sign(beta))

            # Update beta coefficients
            beta -= eta * grad

    return beta

# Simulation parameters
np.random.seed(42)

n = 200  # number of observations
d = 10   # number of features
rho = 3  # noise multiplier for response variable

lambda1, lambda2 = 0.01, 0.01  # Regularization parameters
# Parameters for dynamic ε and δ
gamma = 0.1
zeta = 2.0
eta = 0.001  # Learning rate
iterations = 1000  # Number of iterations for the online algorithm

beta_init = np.random.rand(d)  # Initial beta values
beta_true = np.full((d,), 0.85)  # true regression coefficients all set to 0.85

# Generating synthetic data
X = np.random.normal(0, 1, (n, d))
epsilon_noise = np.random.normal(0, 1, n)
y = X @ beta_true + rho * epsilon_noise

# Fit the model using the revised online algorithm
beta_estimated = canal_adaptive_elastic_net_online(X, y, beta_init, lambda1, lambda2, gamma, zeta, eta, iterations)
predicted = X @ beta_estimated
mse = mean_squared_error(y, predicted)
print("MSE of canal adaptive elastic net:", mse)

# Fit ElasticNet
model_elastic = ElasticNet(alpha=0.1, l1_ratio=0.5)  # alpha is the penalty term, l1_ratio controls the mix of L1 vs L2
model_elastic.fit(X, y)
predicted_elastic = model_elastic.predict(X)

mse_elastic = mean_squared_error(y, predicted_elastic)
print("MSE of elastic net:", mse_elastic)

# Fit Lasso
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X, y)
predicted_lasso = model_lasso.predict(X)
mse_lasso = mean_squared_error(y, predicted_lasso)
print("MSE of lasso:", mse_lasso)

# Fit Ridge
model_ridge = Ridge(alpha=0.1)
model_ridge.fit(X, y)
predicted_ridge = model_ridge.predict(X)
mse_ridge = mean_squared_error(y, predicted_ridge)
print("MSE of ridge:", mse_ridge)

# print("Estimated coefficients:", beta_estimated)


MSE of canal adaptive elastic net: 16.02621327308482
MSE of elastic net: 8.266283970208537
MSE of lasso: 8.290199288553149
MSE of ridge: 8.17960808654734
